In [ ]:
# default_exp USB_video_stream
%load_ext autoreload
%autoreload 2

# FLIR_video_stream
> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import time
import cv2
import PySpin
from threading import Thread
import sys
# import the Queue class from Python 3
if sys.version_info >= (3, 0):
    from queue import Queue
# otherwise, import the Queue class for Python 2.7
else:
    from Queue import Queue

In [ ]:
#export
class USBVideoStream:
    """ Create threaded  USB3 video stream for 1 or more FLIR USB3 cameras, with max queue size
     Pass optional transform code to alter the image before queueing"""
    def __init__(self, transform=None, queue_size=10):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.stopped = False
        self.transform = transform

        # initialize the queue used to store frames read from the video file
        self.Q = Queue(maxsize=queue_size)
        # intialize thread
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True

        self.system = PySpin.System.GetInstance()  # Retrieve singleton reference to system object
        version = self.system.GetLibraryVersion()  # Get current library version
        print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

        self.camlist = self.system.GetCameras()  # Retrieve list of cameras from the system

        # num_cameras =
        # for cam in cam_list: cam.DeInit()
        # system.ReleaseInstance()

        print(f'Number of cameras detected: {self.camlist.GetSize()}')

    def __del__(self):
        self.camlist.Clear()
        self.system.ReleaseInstance()

    def set_cam_mode_continuous(self, i, cam):
        """For all cameras set the acquisition mode set to continuous"""
        node_acquisition_mode = PySpin.CEnumerationPtr(cam.GetNodeMap().GetNode('AcquisitionMode'))
        if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous . Aborting... \n')
            return False

        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(
                node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous , Aborting... \n')
            return False

        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)
        print('Camera acquisition mode set to continuous...')
        return True

    def start(self):
        """ 
        For all cameras,  Initialise and set camera mode to continuous.
        and begin acquisisition
        """
        for i, cam in enumerate(self.camlist): cam.Init()  # intialize cameras

        for i, cam in enumerate(self.camlist):

            # Set acquisition mode to continuous
            self.set_cam_mode_continuous(i, cam)

            # Begin acquiring images
            cam.BeginAcquisition()

            print('Camera %d started acquiring images...' % i)
            print()
        # start a thread to read frames from the file video stream
        self.thread.start()
        return self

    def update(self):
        # keep looping infinitely
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                break

            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                for i, cam in enumerate(self.camlist):
                    try:
                        frame = cam.GetNextImage()
                        image_converted = frame.Convert(PySpin.PixelFormat_BGR8)
                        # image_converted = frame.Convert(PySpin.PixelFormat_Mono8, PySpin.HQ_LINEAR)
                        image_converted = image_converted.GetNDArray()
                        # print(f'Image retrieved from cam {i}, shape = {image_converted.shape}')
                        frame.Release()  # Release image producer/consumer queues since this one was generally
                                         # idle grabbing frames.
                        if self.transform:
                            image_converted = self.transform(image_converted)

                        # add the frame to the queue
                        self.Q.put({'cam':i, 'image':image_converted})

                    except PySpin.SpinnakerException as ex:
                        print('Error: %s' % ex)
                        self.stopped = True

            else:
                time.sleep(0.1)  # Rest for 10ms, we have a full queue

        # self.stream.release()
        for cam in self.camlist:
            # End acquisition
            cam.EndAcquisition()
            cam.DeInit()

    def read(self):
        """Return next frame in the queue"""
        return self.Q.get()

    # Insufficient to have consumer use while(more()) which does
    # not take into account if the producer has reached end of
    # file stream.
    def running(self):
        """ Test if thread id running"""
        return self.more() or not self.stopped

    def more(self):
        """Return True if there are still frames in the queue. 
        If stream is not stopped, try to wait a moment"""
        tries = 0
        while self.Q.qsize() == 0 and not self.stopped and tries < 5:
            time.sleep(0.1)
            tries += 1

        return self.Q.qsize() > 1

    def queue_size(self):
        """Return the queue size, ie number of frames"""
        return self.Q.qsize()

    def stop(self):
        """indicate that the thread should be stopped"""
        self.stopped = True
        # wait until stream resources are released (producer thread might be still grabbing frame)
        self.thread.join()

In [ ]:
show_doc(USBVideoStream.set_cam_mode_continuous)
show_doc(USBVideoStream.start)
show_doc(USBVideoStream.more)
show_doc(USBVideoStream.read)
show_doc(USBVideoStream.queue_size)
show_doc(USBVideoStream.stop)


<h4 id="USBVideoStream.set_cam_mode_continuous" class="doc_header"><code>USBVideoStream.set_cam_mode_continuous</code><a href="__main__.py#L33" class="source_link" style="float:right">[source]</a></h4>

> <code>USBVideoStream.set_cam_mode_continuous</code>(**`i`**, **`cam`**)

For all cameras set the acquisition mode set to continuous

<h4 id="USBVideoStream.start" class="doc_header"><code>USBVideoStream.start</code><a href="__main__.py#L51" class="source_link" style="float:right">[source]</a></h4>

> <code>USBVideoStream.start</code>()

For all cameras,  Initialise and set camera mode to continuous.
and begin acquisisition

<h4 id="USBVideoStream.more" class="doc_header"><code>USBVideoStream.more</code><a href="__main__.py#L121" class="source_link" style="float:right">[source]</a></h4>

> <code>USBVideoStream.more</code>()

Return True if there are still frames in the queue. 
If stream is not stopped, try to wait a moment

<h4 id="USBVideoStream.read" class="doc_header"><code>USBVideoStream.read</code><a href="__main__.py#L110" class="source_link" style="float:right">[source]</a></h4>

> <code>USBVideoStream.read</code>()

Return next frame in the queue

<h4 id="USBVideoStream.queue_size" class="doc_header"><code>USBVideoStream.queue_size</code><a href="__main__.py#L131" class="source_link" style="float:right">[source]</a></h4>

> <code>USBVideoStream.queue_size</code>()

Return the queue size, ie number of frames

<h4 id="USBVideoStream.stop" class="doc_header"><code>USBVideoStream.stop</code><a href="__main__.py#L135" class="source_link" style="float:right">[source]</a></h4>

> <code>USBVideoStream.stop</code>()

indicate that the thread should be stopped

## Test Example Usage

In [ ]:
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from imutils.video import FPS
import imutils
import time
import cv2
# from FLIRCam.USB_video_stream import *

fvs = USBVideoStream() # .start()
fvs.start()
fps = FPS().start()
width = 1000
height = 750
with FFMPEG_VideoWriter('out.mp4', (width,height), 5.0) as video:
    for i in range(50):
        while not fvs.more():
            # wait until 1 or more camera frames arrive
            pass
    
        frame = fvs.read()
        image = imutils.resize(frame['image'], width=width, height=height)
        # display the size of the queue on the frame
        cv2.putText(image, f"Queue Size: {fvs.Q.qsize()}, Camera: {frame['cam']}, Shape: {image.shape}",
            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        print(':', end='', flush=True)  
        # show the frame and update the FPS counter
        video.write_frame(image)
        cv2.imshow("Cam", image)
        fps.update()
        k = cv2.waitKey(10)
        if k == 27:  # Esc key to stop
            break
        elif k == -1:  # normally -1 returned,so don't print it
            continue
        else:
            print(k)

# do a bit of cleanup
fvs.stop()
fps.stop()
print()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# Not FPS will be slower due to video writing

cv2.destroyAllWindows()

del fvs


Library version: 1.27.0.48
Number of cameras detected: 2
Camera acquisition mode set to continuous...
Camera 0 started acquiring images...

Camera acquisition mode set to continuous...
Camera 1 started acquiring images...

::::::::::::::::::::::::::::::::::::::::::::::::::
[INFO] elasped time: 9.30
[INFO] approx. FPS: 5.37


## Show result video

In [ ]:
import moviepy.editor as mvp
mvp.ipython_display('out.mp4', loop=True)